In [1]:
import sys
sys.path.insert(0, '../src')
pickleFolder = '../PickledData/'
pickleFile = 'preProcessedData.p'
import file_operations



(restaurantTrainDict,restaurantTestDict,laptopTrainDict, laptopTestDict)\
    = file_operations.loadAndGetRawDataFromPickle(pickleFolder, pickleFile)
rst_train_sentences = restaurantTrainDict['sentences']['sentence']
lptp_train_sentences = laptopTrainDict['sentences']['sentence']

all data loaded in python dicts


In [14]:
len(rst_train_sentences[1]['aspectCategories']['aspectCategory'])


2

In [20]:
rst_train_sentences[1]['aspectCategories']

{'aspectCategory': [{'@category': 'food', '@polarity': 'positive'},
  {'@category': 'anecdotes/miscellaneous', '@polarity': 'negative'}]}

In [37]:
one_category_c = 0
one_cate_one_asp = 0
one_cate_multi_asp = 0

multi_cate = 0
multi_cate_one_asp = 0
multi_cate_multi_asp = 0
for sent in rst_train_sentences:
    categories = sent['aspectCategories']['aspectCategory']
    if type(categories) is dict:
        one_category_c += 1
        if 'aspectTerms' in sent:
            # print(sent['aspectTerms'])
            aspects = sent['aspectTerms']['aspectTerm']
            if type(aspects) is dict:
                one_cate_one_asp += 1
            else:
                one_cate_multi_asp += 1
    elif type(categories) is list:
        # print(categories)
        multi_cate += 1
        if 'aspectTerms' in sent:
            # print(sent['aspectTerms'])
            aspects = sent['aspectTerms']['aspectTerm']
            if type(aspects) is dict:
                multi_cate_one_asp += 1
            else:
                multi_cate_multi_asp += 1
        
print("one category: {0}".format(one_category_c))
print('one category and one aspect: {0}'.format(one_cate_one_asp))
print('one category and multi aspects: {0}'.format(one_cate_multi_asp))
print('one category and No aspect: {0}'.format(one_category_c - one_cate_one_asp - one_cate_multi_asp))

print('multi categories: {0}'.format(multi_cate))
print('multi categories and one aspect: {0}'.format(multi_cate_one_asp))
print('multi categories and multi aspects: {0}'.format(multi_cate_multi_asp))
print('multi categories and No aspect: {0}'.format(multi_cate - multi_cate_one_asp - multi_cate_multi_asp))






one category: 2468
one category and one aspect: 886
one category and multi aspects: 593
one category and No aspect: 989
multi categories: 576
multi categories and one aspect: 138
multi categories and multi aspects: 406
multi categories and No aspect: 32


In [22]:
counter

2468

In [33]:
rst_train_sentences[4]['aspectTerms']['aspectTerm']

KeyError: 'aspectTerms'

In [38]:
def put_asp_cate(aspect, category, collector):
    """
    
    """
    if aspect in collector:
        cate_dict = collector[aspect]
        if category in cate_dict:
            cate_dict[category] += 1
        else:
            cate_dict[category] = 1
    else:
        collector[aspect] = {}
        collector[aspect][category] = 1

In [41]:
asp_category = {}
for sent in rst_train_sentences:
    categories = sent['aspectCategories']['aspectCategory']
    if type(categories) is dict:
        if 'aspectTerms' in sent:
            
            aspects = sent['aspectTerms']['aspectTerm']
            if type(aspects) is dict:
#                 print(aspects)
                put_asp_cate(aspects['@term'], categories['@category'], asp_category)
            else:
                for asp in aspects:
                    put_asp_cate(asp['@term'], categories['@category'], asp_category)
    # contains multiple categories
    elif type(categories) is list:
        multi_cate += 1
        if 'aspectTerms' in sent:
            # print(sent['aspectTerms'])
            aspects = sent['aspectTerms']['aspectTerm']
            if type(aspects) is dict:
                for cate in categories:
                    put_asp_cate(aspects['@term'], cate['@category'], asp_category)
            else:
                for cate in categories:
                    for asp in aspects:
                        put_asp_cate(asp['@term'], cate['@category'], asp_category)


In [45]:
asp_category['menu']

{'ambience': 6,
 'anecdotes/miscellaneous': 8,
 'food': 50,
 'price': 5,
 'service': 9}

In [43]:
len(asp_category)

1295

In [50]:
asp2category = {}
for asp in asp_category:
    categories = list(asp_category[asp].keys())
    most_freq_category = categories[0]
    for cate in categories:
        if asp_category[asp][cate] > asp_category[asp][most_freq_category]:
            most_freq_category = cate
    asp2category[asp] = most_freq_category


In [53]:
asp2category

{'specials menus': 'food',
 'choice': 'food',
 'dhal': 'food',
 'glasses of wine': 'food',
 'pot-stickers': 'food',
 'tiramisu': 'food',
 'Indian food': 'food',
 'Saul': 'food',
 'house salad': 'food',
 'truffle oil': 'food',
 'apples': 'food',
 'Lobster Bisque': 'food',
 'packed': 'food',
 'mimosas': 'service',
 'red wine': 'service',
 'chicken parm': 'food',
 'chicken vindaloo': 'food',
 'dining rooms': 'ambience',
 'BBQ Salmon': 'food',
 'salads': 'food',
 'presentation': 'food',
 'Grilled Chicken special with Edamame Puree': 'food',
 'beverage selections': 'food',
 'mussels': 'food',
 'extra virgnin olive oil': 'food',
 'plain slice': 'food',
 'eat family style': 'food',
 'tiramisu chocolate cake': 'food',
 'buffet': 'food',
 'White Chocolate Bread Pudding with Gelato and hot chocolate': 'food',
 'dimsum': 'food',
 'signs': 'food',
 'shell crab': 'food',
 'glass of water': 'food',
 'check': 'service',
 'variety of fish': 'food',
 'after dinner drink': 'food',
 'snacking': 'ambience

In [56]:
for sent in rst_train_sentences:
    categories = sent['aspectCategories']['aspectCategory']
    if type(categories) is dict:
        if 'aspectTerms' not in sent:
            print(sent['text'])
            print(categories['@category'])

Where Gabriela personaly greets you and recommends you what to eat.
service
For those that go once and don't enjoy it, all I can say is that they just don't get it.
anecdotes/miscellaneous
I would definitely recommend Mary's and am making it one of my regular neighborhood haunts.
anecdotes/miscellaneous
My Girlfriend and I stumbled onto this hopping place the other night and had a great time!
anecdotes/miscellaneous
He can't help, they're bought up so fast.
anecdotes/miscellaneous
Secondly, on this night the place was overwhelmed by upper east side ladies perfume.
ambience
i have eaten here a handful of times, for no reason besides sheer convenience.
anecdotes/miscellaneous
Obviously run by folks who know a pie.
service
We were very disappointed.
anecdotes/miscellaneous
I really liked this place.
anecdotes/miscellaneous
I will be going back very soon.
anecdotes/miscellaneous
It was pretty inexpensive too.
price
Will not be back.
anecdotes/miscellaneous
This isn't a value joint.
price
L